This section seeks to provide a ranking of how efficiently teams spend their money on a yearly basis. These values will be computed by using a model to predict the salary of a given player on a given team, in a given year. This salary will be compared with the player's actual salary and the difference between the actual and the predicted will tell us if someone is overpaid or underpaid. We will then sum all of these values up for a given team and this will tell us the total amount that they are overpaying or underpaying by. We can then divide this by the total amount spent on salaries, so that we can see overpay or underpay as a percentage of total salary spent, which should make it easier to compare these values across teams and across time.

In [2]:
# Setup autoreload
%load_ext autoreload
%autoreload 2

# Give yourself access to common
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
# Load in neural network for prediction
from common import *
# model = get_rf()
model = get_nn()
nba_data = get_baseline_data("../data/cleaned_data/base_cleaned.csv")
# nba_data.loc[nba_data['playerName'] == 'Michael Jordan'][['playerName','seasonStartYear', 'team']]


c:\Users\mmart\Programming\EECS545\project\EECS-545---Predicting-NBA-Player-s-Salary\common.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba['inflationAdjSalary_log'] = nba['inflationAdjSalary'].apply(lambda x: np.log(x))


In [4]:
# Compute the difference between the amount a team is predicted to spend v. the amount that they actually spent
COL_START_YEAR = 'seasonStartYear'
COL_TEAM = 'team'
COL_PLAYER_NAME = 'playerName'
COL_SALARY = 'inflationAdjSalary'
for year in pd.unique(nba_data[COL_START_YEAR]) :
    for team in pd.unique(nba_data.loc[nba_data[COL_START_YEAR] == year][COL_TEAM]) :
        team_year_player_data = nba_data.loc[(nba_data[COL_TEAM] == team) & (nba_data[COL_START_YEAR] == year)]
        actual_pred_dif = 0
        total_salary_spend = 0
        pred_total_salary_spend = 0
        for i, player in team_year_player_data.iterrows() :
            total_salary_spend += player[COL_SALARY]
            x = (player[get_base_features()].values).reshape(1,-1)
            pred_val = model.predict(x) # Not taking the logarithm seems to be working much better
            pred_total_salary_spend += pred_val
            # print(pred_val)
            actual_pred_dif += player[COL_SALARY] - pred_val
        print("In ", year, " team ", team, " spent ", total_salary_spend, " but should have only spent ", pred_total_salary_spend[0])
        print("Dif: ", total_salary_spend - pred_total_salary_spend[0])
    break


ValueError: could not convert string to float: 'G-F'